# Unit test

In [51]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [83]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection.model_selection import _evaluate_grid_hyperparameters



# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection


metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
#metrics = [add_y_train_argument(metric) for metric in metrics]




In [98]:
def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 100
    param_grid = {
        "alpha": np.logspace(-3, 3, 2),
    }
    lags_grid = [3, 5, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters(
        forecaster         = forecaster,
        y                  = y_feature_selection,
        exog               = exog_feature_selection,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters(
        forecaster         = forecaster,
        y                  = y_feature_selection,
        exog               = exog_feature_selection,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )

    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


forecasters = [
    ForecasterAutoreg(regressor=Ridge(random_state=678), lags=3),
    ForecasterAutoregDirect(regressor=Ridge(random_state=678), lags=3, steps=1),
    ForecasterAutoreg(
        regressor=Ridge(random_state=678),
        lags=3,
        #transformer_y=StandardScaler(),
        transformer_exog=StandardScaler(),
    ),
]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

0
1
2


AssertionError: DataFrame.iloc[:, 3] (column name="mean_absolute_error") are different

DataFrame.iloc[:, 3] (column name="mean_absolute_error") values are different (100.0 %)
[index]: [0, 1, 2, 3, 4, 5]
[left]:  [0.0005606372165538498, 0.0005711071213165906, 0.000598871299685834, 49.21313141756098, 49.55171923705793, 50.51903847355794]
[right]: [0.0007645824801421455, 0.0007750281610598293, 0.0008082819061429347, 50.293556578595144, 50.60648358600952, 51.59721196976838]
At positional index 0, first diff: 0.0005606372165538498 != 0.0007645824801421455

In [109]:
X, y = forecaster.create_train_X_y(y=y_feature_selection, exog=exog_feature_selection)
X_train, y_train, X_test, y_test = forecaster._train_test_split_one_step_ahead(y=y_feature_selection, exog=exog_feature_selection, initial_train_size=100)
X_one_step_ahead = pd.concat([X_train, X_test])
y_one_step_ahead = pd.concat([y_train, y_test])

pd.testing.assert_frame_equal(X, X_one_step_ahead)
pd.testing.assert_series_equal(y, y_one_step_ahead)

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,exog_0,exog_1,exog_2,exog_3,exog_4
2020-01-01 07:00:00,24.995108,54.203926,37.674479,-20.809688,88.677077,36.352492,-0.158530,-1.121145,1.590948,0.279970,0.618352,0.595441
2020-01-01 08:00:00,59.649762,24.995108,54.203926,37.674479,-20.809688,88.677077,36.352492,-0.217728,0.550228,-0.849106,2.073872,0.841560
2020-01-01 09:00:00,-36.402938,59.649762,24.995108,54.203926,37.674479,-20.809688,88.677077,0.340883,-1.159585,-0.991874,-0.807768,0.833413
2020-01-01 10:00:00,-89.906870,-36.402938,59.649762,24.995108,54.203926,37.674479,-20.809688,0.236108,-0.275506,0.094968,1.410528,1.350831
2020-01-01 11:00:00,-0.531842,-89.906870,-36.402938,59.649762,24.995108,54.203926,37.674479,-0.675685,-2.892512,-0.253320,-1.886690,1.007638


,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,exog_0,exog_1,exog_2,exog_3,exog_4
2020-01-01 07:00:00,24.995108,54.203926,37.674479,-20.809688,88.677077,36.352492,-0.158530,-1.121145,1.590948,0.279970,0.618352,0.595441
2020-01-01 08:00:00,59.649762,24.995108,54.203926,37.674479,-20.809688,88.677077,36.352492,-0.217728,0.550228,-0.849106,2.073872,0.841560
2020-01-01 09:00:00,-36.402938,59.649762,24.995108,54.203926,37.674479,-20.809688,88.677077,0.340883,-1.159585,-0.991874,-0.807768,0.833413
2020-01-01 10:00:00,-89.906870,-36.402938,59.649762,24.995108,54.203926,37.674479,-20.809688,0.236108,-0.275506,0.094968,1.410528,1.350831
2020-01-01 11:00:00,-0.531842,-89.906870,-36.402938,59.649762,24.995108,54.203926,37.674479,-0.675685,-2.892512,-0.253320,-1.886690,1.007638


In [112]:
initial_train_size = 100
X, y = forecaster.create_train_X_y(y=y_feature_selection, exog=exog_feature_selection)
X_train, y_train, X_test, y_test = forecaster._train_test_split_one_step_ahead(y=y_feature_selection, exog=exog_feature_selection, initial_train_size=100)

pd.testing.assert_frame_equal(X.iloc[:initial_train_size, :], X_train)
pd.testing.assert_series_equal(y.iloc[:initial_train_size], y_train)

AssertionError: DataFrame are different

DataFrame shape mismatch
[left]:  (100, 12)
[right]: (93, 12)

## _evaluate_grid_hyperparameters: backtesting vs one-step-ahead

In [61]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from copy import copy
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from skforecast.datasets import fetch_dataset
from skforecast.plot import set_dark_theme
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries

# Warnings
# ==============================================================================
import warnings
from skforecast.exceptions import IgnoredArgumentWarning
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', category=IgnoredArgumentWarning)

In [62]:
from skforecast.model_selection.model_selection import _evaluate_grid_hyperparameters
from sklearn.model_selection import ParameterGrid

In [92]:
# Data
# ==============================================================================
data = fetch_dataset(
    name="h2o_exog",
    verbose=False,
)
data = data.sort_index()
end_train = "2001-01-01 23:59:00"
end_validation = "2006-01-01 23:59:00"
target = "y"
exog_features = ['exog_1', 'exog_2']

In [97]:
forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(random_state=123, verbose=-1),
                lags      = 10,
                transformer_y=StandardScaler(),
            )
lags_grid = [3, 10, [1, 2, 3, 20]]
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [2, 3, 5]
}
param_grid = list(ParameterGrid(param_grid))

results_backtesting = _evaluate_grid_hyperparameters(
    forecaster         = forecaster,
    y                  = data.loc[:end_validation, target],
    exog               = data.loc[:end_validation, exog_features] if exog_features else None,
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    steps              = 1,
    method             = 'backtesting',
    refit              = False,
    metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_scaled_error'],
    initial_train_size = len(data.loc[:end_train]),
    fixed_train_size   = False,
    return_best        = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = False
)

results_one_step_ahead = _evaluate_grid_hyperparameters(
    forecaster         = forecaster,
    y                  = data.loc[:end_validation, target],
    exog               = data.loc[:end_validation, exog_features] if exog_features else None,
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_scaled_error'],
    initial_train_size = len(data.loc[:end_train]),
    method             = 'one_step_ahead',
    return_best        = False,
    verbose            = False,
    show_progress      = False
)


pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)

[0.86050004 0.62134986 0.92298243 0.76921168 0.80404391 0.85138085
 0.8390067  0.8544545  0.86050004 0.86050004 0.86050004 0.86050004
 0.86050004 0.67881567 0.83968142 0.86732965 0.81633951 0.86234101
 0.92600178 0.8544545  0.86050004 0.86050004 0.86050004 0.86050004
 0.86050004 0.67881567 0.83968142 0.78516049 0.82152907 0.86747834
 0.92600178 0.8544545  0.86050004 0.86050004 0.86050004 0.86050004
 0.86050004 0.77849741 0.92690999 0.92086445 0.8390067  0.92600178
 0.8544545  0.86050004 0.86050004 0.86050004 0.86050004 0.86050004
 0.86050004 0.73771259 0.83968142 0.78516049 0.82152907 0.87246699
 0.8544545  0.86050004 0.86050004 0.86050004 0.86050004 0.86050004]
[0.8727035  0.63886949 0.94836765 0.76479828 0.79310052 0.84657345
 0.8337     0.84016863 0.8727035  0.8727035  0.8727035  0.8727035
 0.8727035  0.6963353  0.87220827 0.87540656 0.81733456 0.8564396
 0.94192984 0.84016863 0.8727035  0.8727035  0.8727035  0.8727035
 0.8727035  0.6963353  0.87220827 0.7932374  0.81012869 0.860254

AssertionError: DataFrame.iloc[:, 0] (column name="lags") are different

DataFrame.iloc[:, 0] (column name="lags") values are different (33.33333 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[left]:  [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3], [1, 2, 3, 20], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]
[right]: [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3, 20], [1, 2, 3, 20], [1, 2, 3], [1, 2, 3, 20]]
At positional index 9, first diff: [ 1  2  3 20] !=

## _calculate_metrics_multiseries vs _calculate_metrics_multiseries_one_step_ahead

In [29]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [30]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [31]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

X, y = forecaster.create_train_X_y(series=data)
X_train = X.loc[X.index <= end_train, :]
y_train = y.loc[y.index <= end_train]
X_test = X.loc[X.index > end_train, :]
y_test = y.loc[y.index > end_train]

forecaster.regressor.fit(X_train, y_train)
pred = forecaster.regressor.predict(X_test)

In [32]:
# Test results _calculate_metrics_multiseries_one_step_ahead and _calculate_metrics_multiseries
# ==============================================================================
# Metrics should be equal when using step=1 in backtesting

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

if forecaster.encoding in ['ordinal', 'ordinal_category']:
    X_train_encoding = forecaster.encoder.inverse_transform(X_train[['_level_skforecast']]).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(X_test[['_level_skforecast']]).ravel()
elif forecaster.encoding == 'onehot':
    X_train_encoding = forecaster.encoder.inverse_transform(
                            X_train.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(
                            X_test.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
X_train_encoding = pd.Series(X_train_encoding, index=X_train.index)
X_test_encoding = pd.Series(X_test_encoding, index=X_test.index)

metrics_one_step_ahead = _calculate_metrics_multiseries_one_step_ahead(
    y_true = y_test,
    y_pred = pred,
    y_pred_encoding = X_test_encoding,
    y_train = y_train,
    y_train_encoding = X_train_encoding,
    levels = ['item_1', 'item_2', 'item_3'],
    metrics = metrics,
    add_aggregated_metric = True
)

display(metrics_one_step_ahead)

# Metrics with _calculate_metrics_multiseries
folds = _create_backtesting_folds(
    data = data,
    window_size = 24,
    initial_train_size = len(data_train),
    test_size = 1,
    externally_fitted = False,
    refit = False,
    fixed_train_size = True,
    gap = 0,
    skip_folds = None,
    allow_incomplete_fold = True,
    return_all_indexes = False,
    differentiation = None,
    verbose = False
)
_, predictions = backtesting_forecaster_multiseries(
    series=data,
    forecaster=forecaster,
    steps=1,
    metric=metrics,
    initial_train_size = len(data_train),
    refit=False,
    add_aggregated_metric=True,
    show_progress=False
)
metrics_backtesting = _calculate_metrics_multiseries(
    series = data,
    predictions= predictions,
    folds= folds,
    span_index= data.index,
    window_size = 24,
    metrics= metrics,
    levels= ['item_1', 'item_2', 'item_3'],
    add_aggregated_metric = True
)

display(metrics_backtesting)

assert metrics_backtesting.equals(metrics_one_step_ahead)

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.820748,0.040761,0.537711
1,item_2,2.330964,0.152464,0.989540
2,item_3,3.093253,0.207347,0.835301
3,average,2.081655,0.133524,0.787517
4,weighted_average,2.081655,0.133524,0.787517
5,pooling,2.081655,0.133524,0.823316


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.820748,0.040761,0.537711
1,item_2,2.330964,0.152464,0.989540
2,item_3,3.093253,0.207347,0.835301
3,average,2.081655,0.133524,0.787517
4,weighted_average,2.081655,0.133524,0.787517
5,pooling,2.081655,0.133524,0.823316


## _evaluate_grid_hyperparameters_multiseries: backtesting vs one-step-ahead

In [33]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.metrics import mean_absolute_scaled_error

In [34]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()
exog_features = None

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [35]:
forecaster = ForecasterAutoregMultiSeries(
    regressor          = LGBMRegressor(random_state=123, verbose=-1),
    lags               = 24,
    encoding           = 'ordinal',
    transformer_series = None,
    transformer_exog   = None,
    weight_func        = None,
    series_weights     = None,
    differentiation    = None,
    dropna_from_series = False,
    fit_kwargs         = None,
    forecaster_id      = None
)
lags_grid = [3, 10]
param_grid = {
    'n_estimators': [5, 10],
    'max_depth': [2, 3]
}
param_grid = list(ParameterGrid(param_grid))

results_backtesting = _evaluate_grid_hyperparameters_multiseries(
                            forecaster            = forecaster,
                            series                = data,
                            param_grid            = param_grid,
                            steps                 = 1,
                            metric                = metrics,
                            aggregate_metric      = ['weighted_average', 'average', 'pooling'],
                            initial_train_size    = len(data_train),
                            method                = 'backtesting',
                            fixed_train_size      = True,
                            gap                   = 0,
                            skip_folds            = None,
                            allow_incomplete_fold = True,
                            levels                = None,
                            exog                  = data.loc[:end_validation, exog_features] if exog_features else None,
                            lags_grid             = lags_grid,
                            refit                 = False,
                            n_jobs                = 'auto',
                            return_best           = False,
                            verbose               = False,
                            show_progress         = False,
                            suppress_warnings     = False
                        )

results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
                            forecaster            = forecaster,
                            series                = data,
                            param_grid            = param_grid,
                            metric                = metrics,
                            initial_train_size    = len(data_train),
                            method                = 'one_step_ahead',
                            exog                  = data.loc[:data_train, exog_features] if exog_features else None,
                            lags_grid             = lags_grid,
                            return_best           = False,
                            verbose               = False,
                            show_progress         = False
                        )

assert results_backtesting.equals(results_one_step_ahead)


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1710: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(
